In [21]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from kafka import KafkaProducer
import json
import time
import datetime

import sys
print(sys.executable)

# MySQL Connection
host = '192.168.3.102'
user = 'devturingmx'
password = quote_plus('des@rrollo')
database = 'sistema_monitoreo'

# Create SQLAlchemy engine
connection_url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
engine = create_engine(connection_url)

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

def fetch_and_send_data():
    try:
        query = "SELECT * FROM fgenerador1_data ORDER BY id DESC LIMIT 1"
        with engine.connect() as connection:
            # Añadimos text ya que sqlalchemy no es compatible directamente con cadenas de texto
            result = connection.execute(text(query)).fetchone()
            if result:
                # Modificamos data para convertir manualmete el timestamp por que json.dumps no puede convertirlo automaticamente
                data = {
                key: (value.isoformat() if isinstance(value, (datetime.datetime, datetime.date)) else value)
                for key, value in dict(result._mapping).items()
                }
                
                producer.send('rigcore-data', value=data)
                producer.flush()
                print(f"Sent data: {data}")
    except Exception as e:
        print(f"Error: {e}")

def main():
    while True:
        fetch_and_send_data()
        time.sleep(1)

if __name__ == "__main__":
    main()

/Users/ovillajc/Documents/Desarrollo/python/fast_api/rigcore_simulator/.venv/bin/python
Sent data: {'id': 2440, 'timestamp': '2025-06-01T23:59:00', 'rig_config_id': 1, 'session_timestamp': '2025-06-05T00:00:00', 'channel_01': 96.61, 'name_1': 'Canal 1', 'unit_01': 'g/cm³', 'channel_02': 114.64, 'name_2': 'Canal 2', 'unit_02': 'ft', 'channel_03': 118.33, 'name_3': 'Canal 3', 'unit_03': 'ft', 'channel_04': 130.21, 'name_4': 'Canal 4', 'unit_04': 'l', 'channel_05': 139.7, 'name_5': 'Canal 5', 'unit_05': 'm³/h', 'channel_06': 149.52, 'name_6': 'Canal 6', 'unit_06': '°F', 'channel_07': 157.9, 'name_7': 'Canal 7', 'unit_07': 'm³/h', 'channel_08': 167.34, 'name_8': 'Canal 8', 'unit_08': 'l', 'channel_09': 182.2, 'name_9': 'Canal 9', 'unit_09': '°F', 'channel_10': 188.56, 'name_10': 'Canal 10', 'unit_10': 'scaled', 'channel_11': 198.65, 'name_11': 'Canal 11', 'unit_11': 'rpm', 'channel_12': 209.6, 'name_12': 'Canal 12', 'unit_12': 'deg/s', 'channel_13': 216.41, 'name_13': 'Canal 13', 'unit_13'

KeyboardInterrupt: 

In [22]:
from kafka import KafkaConsumer
import json
import time

# Configuracion del consumidor
consumer = KafkaConsumer(
    'rigcore-data',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    enable_auto_commit=True,     
    group_id='rigcore-consumer-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("Escuchando mensajes en el topic 'rigcore-data'...")

try:
    for message in consumer:
        data = message.value
        print(f"Mensaje recibido: {data}")
        time.sleep(1)
except KeyboardInterrupt:
    print("\nLectura interrumpida por el usuario.")
finally:
    consumer.close()
    print("Consumer cerrado correctamente.")

Escuchando mensajes en el topic 'rigcore-data'...
Mensaje recibido: {'id': 2440, 'timestamp': '2025-06-01T23:59:00', 'rig_config_id': 1, 'session_timestamp': '2025-06-05T00:00:00', 'channel_01': 96.61, 'name_1': 'Canal 1', 'unit_01': 'g/cm³', 'channel_02': 114.64, 'name_2': 'Canal 2', 'unit_02': 'ft', 'channel_03': 118.33, 'name_3': 'Canal 3', 'unit_03': 'ft', 'channel_04': 130.21, 'name_4': 'Canal 4', 'unit_04': 'l', 'channel_05': 139.7, 'name_5': 'Canal 5', 'unit_05': 'm³/h', 'channel_06': 149.52, 'name_6': 'Canal 6', 'unit_06': '°F', 'channel_07': 157.9, 'name_7': 'Canal 7', 'unit_07': 'm³/h', 'channel_08': 167.34, 'name_8': 'Canal 8', 'unit_08': 'l', 'channel_09': 182.2, 'name_9': 'Canal 9', 'unit_09': '°F', 'channel_10': 188.56, 'name_10': 'Canal 10', 'unit_10': 'scaled', 'channel_11': 198.65, 'name_11': 'Canal 11', 'unit_11': 'rpm', 'channel_12': 209.6, 'name_12': 'Canal 12', 'unit_12': 'deg/s', 'channel_13': 216.41, 'name_13': 'Canal 13', 'unit_13': 'l/min', 'channel_14': 229.19